In [ ]:
## Update matplotlib to a version that can label bar graphs.
#!pip install -U matplotlib --user

In [ ]:
%matplotlib inline

import sympy as sp
sp.init_printing(use_latex ='mathjax')
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lmfit as lf
import os
import seaborn as sns
from myheatmap import myheatmap
from lmfit import Model
import lmfit
from tabulate import tabulate
import matplotlib

verbose = False

sns.set_context('talk')

matplotlib.__version__ ## Need version 3.5 for labeled bargraphs. otherwise set labelbarchart = False below

In [ ]:
#Define all variables

#individual springs that correspond to individual masses
k1 = sp.symbols('k_1', real = True)
k2 = sp.symbols('k_2', real = True)

#springs that connect two masses
k12 = sp.symbols('k_12', real = True)

#damping coefficients
b1 = sp.symbols('b1', real = True)
b2 = sp.symbols('b2', real = True)
 
#masses
m1 = sp.symbols('m1', real = True)
m2 = sp.symbols('m2', real = True)

#Driving force amplitude
F = sp.symbols('F', real = True)

#driving frequency (leave as variable)
wd = sp.symbols('\omega_d', real = True)

In [ ]:
def syserror(x_found,x_set):
    return abs(x_found-x_set)/x_set

def combinedsyserror(syserrors, notdof): # notdof = not degrees of freedom, meaning the count of fixed parameters.
    dof = len(syserrors) - notdof
    squared = [(err**2) for err in syserrors]
    rms = np.sqrt(sum(squared) / dof)
    avg = sum(syserrors)/ dof
    return avg, rms

def complexamp(A,phi):
    return A * np.exp(1j*phi)

def rsqrd(model, data, plot=False, x=None, newfigure = True):
    SSres = sum((data - model)**2)
    SStot = sum((data - np.mean(data))**2)
    rsqrd = 1 - (SSres/ SStot)
    
    if plot:
        if newfigure:
            plt.figure()
        plt.plot(x,data, 'o')
        plt.plot(x, model, '--')
    
    return rsqrd

In [ ]:
#Solve for driving amplitudes and phase

#Matrix for complex equations of motion
driven = sp.Matrix([[-wd**2*m1 + 1j*wd*b1 + k1 + k12, -k12], [-k12, -wd**2*m2 + 
  1j*wd*b2 + k2 + k12]])

#Matrices for Cramer's Rule
driven_m1 = sp.Matrix([[F, -k12], [0, -wd**2*m2 + 1j*wd*b2 + k2 + k12]])

driven_m2 = sp.Matrix([[-wd**2*m1 + 1j*wd*b1 + k1 + k12, F], [-k12, 0]])

#Apply Cramer's Rule
complexamp1, complexamp2 = (driven_m1.det()/driven.det(), driven_m2.det()/driven.det())

#Solve for phases for each mass
delta1 = sp.arg(complexamp1) # Returns the argument (phase angle in radians) of a complex number. 
delta2 = sp.arg(complexamp2) # sp.re(complexamp2)/sp.cos(delta2) (this is the same thing)

#Wrap phases for plots

wrap1 = (delta1)%(2*sp.pi)
wrap2 = (delta2)%(2*sp.pi)

#Solve for amplitude coefficients
amp1 = sp.Abs(complexamp1)
amp2 = sp.Abs(complexamp2)

"""complexamp1 = amp1 * sp.exp(sp.I * sp.pi * delta1)
complexamp2 = amp2 * sp.exp(sp.I * sp.pi * delta2) """

if verbose:
    display(r"R1 Amplitude:")
    display(amp1)

    display(r"R2 Amplitude:")
    display(amp2)

    display(r"R1 complex amplitude:")
    display(complexamp1)
    
    display(r"R2 complex amplitude:")
    display(complexamp2)
    
    display("R1 Real amplitude:")
    display(sp.re(complexamp1))
    
    display("R1 Imaginary amplitude:")
    display(sp.im(complexamp1))
    
    display("R2 Real amplitude:")
    display(sp.re(complexamp2))
    
    display("R2 Imaginary amplitude:")
    display(sp.im(complexamp2))

#lambdify curves

c1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), amp1)
t1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F,  m1, m2), wrap1)

c2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), amp2)
t2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), wrap2)

re1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.re(complexamp1))
im1 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.im(complexamp1))
re2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.re(complexamp2))
im2 = sp.lambdify((wd, k1, k2, k12, b1, b2, F, m1, m2), sp.im(complexamp2))

#define functions

#curve = amplitude, theta = phase, e = error (i.e. noise)
def curve1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return c1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def theta1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return t1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) - 2*np.pi + e
    
def curve2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return c2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def theta2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return t2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) - 2*np.pi + e
    
def realamp1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return re1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def imamp1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return im1(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def realamp2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return re2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e
    
def imamp2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2, e):
     return im2(w, k_1, k_2, k_12, b1_, b2_, F_, m_1, m_2) + e

In [ ]:
#Use functions to make matrices of amplitude and phase for each resonator (with noise)

#define set values
k1_set = 12
k2_set = 27
k12_set = 1
b1_set = 1
b2_set = 0.5
F_set = 1
m1_set = 5
m2_set = 3

vals_set = [m1_set, m2_set, b1_set, b2_set, k1_set, k2_set, k12_set, F_set]

#define number of samples (i.e. range over which to vary the frequency)
n = 50

noiselevel = .01
use_complexnoise = False

amplitudenoisefactor1 = 0.005
amplitudenoisefactor2 = 0.0005
phasenoisefactor1 = 0.1
phasenoisefactor2 = 0.2
complexamplitudenoisefactor = 0.0005

#define noise (randn(n,) gives a array of normally-distributed random numbers of size n)
amp1_noise = noiselevel* amplitudenoisefactor1 * np.random.randn(n,)
phase1_noise = noiselevel* phasenoisefactor1 * np.random.randn(n,)
amp2_noise = noiselevel* amplitudenoisefactor2 * np.random.randn(n,)
phase2_noise = noiselevel* phasenoisefactor2 * np.random.randn(n,)
complex_noise = noiselevel* complexamplitudenoisefactor * np.random.randn(n,)

#define driving frequency range (gives array of n evenly spaced numbers between 0.1 and 5)
drive = np.linspace(0.1, 5, num = n)

R1_amp_noiseless = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R1_phase_noiseless = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_amp_noiseless = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_phase_noiseless = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)

R1_real_amp_noiseless = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R1_im_amp_noiseless = imamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_real_amp_noiseless = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)
R2_im_amp_noiseless = imamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0)

def amp(a,b):
    return np.sqrt(a**2 + b**2)

usenoise = True

# ***
if usenoise: # add a random vector of positive and negative numbers to the curve.
  
    if use_complexnoise: # apply noise in cartesian coordinates
        R1_real_amp = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise)
        R1_im_amp   = imamp1  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise)
        R2_real_amp = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise)
        R2_im_amp   = imamp2  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, complex_noise)
        
        R1_amp   = amp(R1_real_amp, R1_im_amp)
        R2_amp   = amp(R2_real_amp, R2_im_amp)
        R1_phase = np.unwrap(np.angle(R1_real_amp + R1_im_amp*1j))
        R2_phase = np.unwrap(np.angle(R2_real_amp + R2_im_amp*1j))
                 
    else: # apply noise in polar coordinates
        R1_amp   = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, amp1_noise)
        R1_phase = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, phase1_noise)
        R2_amp   = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, amp2_noise)
        R2_phase = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, phase2_noise)
        
        R1_complexamp = complexamp(R1_amp, R1_phase)
        R2_complexamp = complexamp(R2_amp, R2_phase)
        
        R1_real_amp = np.real(R1_complexamp)
        R1_im_amp   = np.imag(R1_complexamp)
        R2_real_amp = np.real(R2_complexamp)
        R2_im_amp   = np.imag(R2_complexamp)
        
else:
    R1_amp = R1_amp_noiseless
    R1_phase = R1_phase_noiseless
    R2_amp = R2_amp_noiseless
    R2_phase = R2_phase_noiseless
    R1_real_amp = R1_real_amp_noiseless
    R1_im_amp = R1_im_amp_noiseless
    R2_real_amp = R2_real_amp_noiseless
    R2_im_amp = R2_im_amp_noiseless
    
def listlength(list1):
    try:
        length = len(list1)
    except TypeError:
        length = 1
    return length

if use_complexnoise:
    # Unfortunately doesn't yet work with multiple drive frequencies
    def noisyR1ampphase(drive):       
        n = listlength(drive)
        ## calculate fresh the noise of amplitude 1. This is an independent noise calculation.
        R1_real_amp = realamp1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R1_im_amp   = imamp1  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R1complexamp = R1_real_amp + 1j * R1_im_amp
        return amp(R1_real_amp,R1_im_amp), np.angle(R1_real_amp + R1_im_amp*1j), R1complexamp

    def noisyR2ampphase(drive):
        n = listlength(drive)
        ## calculate fresh the noise of amplitude. This is an independent noise calculation.
        R2_real_amp = realamp2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R2_im_amp   = imamp2  (drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* complexamplitudenoisefactor * np.random.randn(n,))
        R2complexamp = R2_real_amp + 1j * R2_im_amp
        return amp(R2_real_amp,R2_im_amp), np.angle(R2_real_amp + R2_im_amp*1j), R2complexamp

else:
    def noisyR1ampphase(drive):
        n = listlength(drive)
        a = curve1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* amplitudenoisefactor1 * np.random.randn(n,))
        p = theta1(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* phasenoisefactor1 * np.random.randn(n,))
        return a,p, complexamp(a,p)

    def noisyR2ampphase(drive):
        n = listlength(drive)
        a = curve2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* amplitudenoisefactor2 * np.random.randn(n,))
        p = theta2(drive, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, noiselevel* phasenoisefactor2 * np.random.randn(n,))
        return a,p, complexamp(a,p)
    
res1 = np.sqrt(k1_set/m1_set)
res2 = np.sqrt(k2_set/m2_set)

print('Resonant frequencies expected at approximately ' 
      + str(res1) + ' and ' 
      + str(res2))

In [ ]:
#Choose points for SVD

desiredfreqs = [res1, res2]

p = []
for f in desiredfreqs:
        absolute_val_array = np.abs(drive - f)
        f_index = absolute_val_array.argmin()
        p.append(f_index)

#p = [14,15,16,29,30,31] #indices of frequencies to use
#p=[14,16]
#p = range(len(drive))
print(p)

def SNRcalc(freq, plot = False, ax = None):
    n = 50 # number of randomized values to calculate
    amps1 = np.zeros(n)
    zs1 = np.zeros(n ,dtype=complex)
    amps2 = np.zeros(n)
    zs2 = np.zeros(n ,dtype=complex)
    for j in range(n):
        thisamp1, _, thisz1 = noisyR1ampphase(freq)
        amps1[j] = thisamp1
        zs1[j] = thisz1[0]
        thisamp2, _, thisz2 = noisyR2ampphase(freq)
        amps2[j] = thisamp2
        zs2[j] = thisz2[0]
    SNR_R1 = np.mean(amps1) / np.std(amps1)
    SNR_R2 = np.mean(amps2) / np.std(amps2)

    if plot:
        if ax is not None:
            plt.sca(ax)
        plt.plot(np.real(zs), np.imag(zs), '.', alpha = .2)
        plt.plot(0,0, 'o')
        plt.gca().axis('equal');
        plt.title('Freq: ' + str(freq) + ', SNR: ' + '{num:.{dig}g}'.format(num=SNR, dig=3))
    return SNR_R1,SNR_R2


table = []
for i in range(len(p)):
    SNR1, SNR2 = SNRcalc(drive[p[i]])
    table.append([drive[p[i]], R1_amp[p[i]], R1_phase[p[i]], R2_amp[p[i]], R2_phase[p[i]], 
                  complexamp(R1_amp[p[i]],R1_phase[p[i]] ),
                  complexamp(R2_amp[p[i]], R2_phase[p[i]]),
                  SNR1, SNR2,
                 syserror(R1_amp[p[i]], R1_amp_noiseless[p[i]]),
                 (R1_phase[p[i]] - R1_phase_noiseless[p[i]]),
                 syserror(R2_amp[p[i]],R2_amp_noiseless[p[i]]),
                 (R2_phase[p[i]]-R2_phase_noiseless[p[i]]),
                 ])
                 
    
df = pd.DataFrame(data = table, columns = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase',
                                           'R1AmpCom', 'R2AmpCom',
                                           'SNR_R1','SNR_R2',
                                           'R1Amp_syserror', 'R1Phase_diff', 'R2Amp_syserror', 'R2Phase_diff'])

df

In [ ]:
#Plots of simulated spectra

print('Simulated spectra data, as if from experiment')
print('Resonant frequencies expected at ' 
      + str(res1) + ' and ' 
      + str(res2))

morefrequencies = np.linspace(0.1, 5, num = n*10)

fig, ((ax1, ax3),(ax2,ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (10,10))

ax1.plot(morefrequencies, curve1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), # true curve
         color = 'gray', alpha = 0.2)
ax1.plot(drive, R1_amp, '.', color = 'lightsteelblue') # noisy simulated data
ax1.set_ylabel('Amplitude\n')
ax1.set_title('Simulated R1 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax1.plot(df.drive[i], df.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax2.plot(morefrequencies, theta1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set,  0), # true curve
         color = 'gray', alpha = 0.2)        
ax2.plot(drive, R1_phase, '.', color = 'lightsteelblue') # noisy simulated data
ax2.set_ylabel('Phase (rad)')
ax2.set_title('Simulated R1 Phase')

#For loop to plot chosen values from table
for i in range(df.shape[0]):
    ax2.plot(df.drive[i], df.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax3.plot(morefrequencies, curve2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), # true curve
         color = 'gray', alpha = 0.2)    
ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
ax3.set_ylabel('Amplitude\n')
ax3.set_title('Simulated R2 Amp')

#For loop to plot chosen values from table
for i in range(df.shape[0]):
    ax3.plot(df.drive[i], df.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
    
ax4.plot(morefrequencies, theta2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set,  0), # true curve
         color = 'gray', alpha = 0.2)
ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
ax4.set_ylabel('Phase (rad)')
ax4.set_title('Simulated R2 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax4.plot(df.drive[i], df.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
for ax in [ax1,ax2,ax3,ax4]:
    plt.sca(ax)
    plt.xticks([res1, res2])
    ax.set_xlabel('Freq (rad/s)')

    
plt.tight_layout()

def plotcomplex(complexZ, parameter, title = 'Complex Amplitude', cbar_label='Frequency (rad/s)', label_markers=[], ax=plt.gca(), s=50, cmap = 'magma'):
    plt.sca(ax)
    sc = ax.scatter(np.real(complexZ), np.imag(complexZ), s=s, c = parameter, cmap = cmap ) # s is marker size
    cbar = plt.colorbar(sc)
    cbar.outline.set_visible(False)
    cbar.set_label(cbar_label)
    ax.set_xlabel('Real Amplitude')
    ax.set_ylabel('Imaginary Amplitude')
    ax.axis('equal');
    plt.title(title)
    plt.gcf().canvas.draw()
    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    plt.vlines(0, ymin=ymin, ymax = ymax, colors = 'k', linestyle='solid', alpha = .5)
    plt.hlines(0, xmin=xmin, xmax = xmax, colors = 'k', linestyle='solid', alpha = .5)
    #ax.plot([0,1],[0,0], lw=10,transform=ax.xaxis.get_transform() )#,transform=ax.xaxis.get_transform() ) #transform=ax.transAxes
    
    # label markers that are closest to the desired frequencies
    for label in label_markers:
        absolute_val_array = np.abs(drive - label)
        label_index = absolute_val_array.argmin()
        closest_element = parameter[label_index]
        plt.annotate(text=str(round(closest_element,2)), 
                     xy=(np.real(complexZ[label_index]), np.imag(complexZ[label_index])) )

#Calculate complex amplitudes of spectra

Z1 = complexamp(R1_amp, R1_phase)
Z2 = complexamp(R2_amp, R2_phase)

#fig, (ax5, ax6) = plt.subplots(1, 2, figsize = (10,5))

# true curves
ax5.plot(realamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)
ax6.plot(realamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)

plotcomplex(Z1, drive, 'Complex Amplitude Z1', ax=ax5, label_markers=[res1,res2])
ax5.scatter(np.real(df.R1AmpCom), np.imag(df.R1AmpCom), s=150, facecolors='none', edgecolors='k') 
plotcomplex(Z2, drive, 'Complex Amplitude Z2', ax=ax6, label_markers=[res1,res2])
ax6.scatter(np.real(df.R2AmpCom), np.imag(df.R2AmpCom), s=150, facecolors='none', edgecolors='k') 
      
plt.tight_layout()

In [ ]:
df

In [ ]:
#Singular Value Decomposition

verbose = True

elementnames = ['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']

def Zmatrix2resonators(df, frequencycolumn = 'drive', complexamplitude1 = 'R1AmpCom', complexamplitude2 = 'R2AmpCom', dtype=np.double):
    Zmatrix = []
    for rowindex in df.index:
        w = df[frequencycolumn][rowindex]
        #print(w)
        ZZ1 = df[complexamplitude1][rowindex]
        ZZ2 = df[complexamplitude2][rowindex]
        Zmatrix.append([-w**2*np.real(ZZ1), 0, -w*np.imag(ZZ1), 0, np.real(ZZ1), 0, np.real(ZZ1)-np.real(ZZ2), -1])
        Zmatrix.append([-w**2*np.imag(ZZ1), 0, w*np.real(ZZ1), 0, np.imag(ZZ1), 0, np.imag(ZZ1)-np.imag(ZZ2), 0])
        Zmatrix.append([0, -w**2*np.real(ZZ2), 0, -w*np.imag(ZZ2), 0, np.real(ZZ2), np.real(ZZ2)-np.real(ZZ1), 0])
        Zmatrix.append([0, -w**2*np.imag(ZZ2), 0, w*np.real(ZZ2), 0, np.imag(ZZ2), np.imag(ZZ2)-np.imag(ZZ1), 0])
    #display(Zmatrix)
    return np.array(Zmatrix, dtype=dtype)

Zmatrix = Zmatrix2resonators(df, frequencycolumn = 'drive', complexamplitude1 = 'R1AmpCom', complexamplitude2 = 'R2AmpCom')

if verbose:
    print("Z matrix:") 
    display(pd.DataFrame(Zmatrix, columns = elementnames))

#SVD
u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)
#u, s, vh = sc.linalg.svd(Zmatrix, full_matrices = False, lapack_driver = 'gesvd')

# the smallest singular value is always the last one, index -1
print('Singular values:')
print(s)

plt.semilogy(s, '.')
plt.axhline(10**-2, color='gray')
plt.title('singular values');

In [ ]:
# singular vectors. # M1, M2, B1, B2, K1, K2, K12, FD
#vh

In [ ]:
print("Checking the python-normalization")

vect =vh[-1]
sq = [el**2 for el in vect]
sum(sq)

In [ ]:
[M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1]

vals = [['Norm m1', 'Norm m2', 'Norm b1', 'Norm b2', 'Norm k1', 'Norm k2','Norm c12', 'Norm Driving Force']]
vals.append((M1, M2, B1, B2, K1, K2, K12, FD))

#print("Python-Normalized singular vector corresponding to elements vector")
#print(tabulate(vals, headers = 'firstrow', tablefmt = 'fancy_grid'))

In [ ]:
#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')
print("The python normalization loses one degree of freedom, \
so we must be able to independently determine one element; say it's the driving force")

def normalize_elements_by_force(M1, M2, B1, B2, K1, K2, K12, FD, F_set):
    c = F_set / FD
    elements = (M1 * c, M2 * c, B1 * c, B2 * c, K1 * c, K2 * c, K12 * c, FD * c)
    return elements

elements = normalize_elements_by_force(M1, M2, B1, B2, K1, K2, K12, FD, F_set)
#elements = [element.real for element in elements if element.imag == 0 ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]
avg1d, rms1d = combinedsyserror(syserrors, 1)

vals2 = [['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']]
vals2.append(elements)
vals2.append(vals_set)
vals2.append(syserrors)

M1, M2, B1, B2, K1, K2, K12, FD = elements

#print(tabulate(vals2, headers = 'firstrow', tablefmt = 'fancy_grid'))

#Calculate Rsqrd compared to 1D nullspace to test the 1D nullspace model

plotting = False

amp1_rsqrd = rsqrd(model=curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), data=R1_amp, plot=plotting, x=drive)
phase1_rsqrd = rsqrd(data=R1_phase, model=theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
amp2_rsqrd = rsqrd(data=R2_amp, model=curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
phase2_rsqrd = rsqrd(data = R2_phase, model= theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)

print("How good are the 1D nullspace results?")

## Calculate systematic error to test the 1D nullspace results

elementsdf = pd.DataFrame(vals2, index = ['labels', 'from_svd', 'set', "syserror"])
new_header = elementsdf.iloc[0] 
elementsdf.columns = new_header

display(elementsdf)
print('Noise level: ' + str(noiselevel))
print('R1 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R1.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R1.max(), dig=3))
print('R2 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R2.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R2.max(), dig=3))
print('Average systematic error: ', '{num:.{dig}g}'.format(num=avg1d, dig=3))
print('RMS systematic error: ', '{num:.{dig}g}'.format(num=rms1d, dig=3))

labelbarchart = False

fig,ax = plt.subplots()
if labelbarchart:
    bargraph = ax.bar(elementsdf.columns,elementsdf.transpose()['syserror']*100)
    plt.gca().bar_label(bargraph)
else:
    (elementsdf.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');
plt.title('1D nullspace')
plt.show()

print("R1 Amp Rsqrd = ", amp1_rsqrd,
     "\nR1 Phase Rsqrd = ", phase1_rsqrd,
      "\nR2 Amp Rsqrd = ", amp2_rsqrd,
      "\nR2 Phase Rsqrd = ", phase2_rsqrd)

In [ ]:
## what if the null-space is 2D?

#View vh array and assign variables to proper row vector

print('Results of analysis versus set values:')
print("If the null-space is 2D, we must be able to independently determine two elements; say it's m1 and m2")
## but it would be cool if it were k1/m1 and k2/m2 because that should be known from the measured resonant frequencies

def elements_assuming_2d(vh):
    # elements vector: 'm1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force'
    vect1 = vh[-1]
    vect2 = vh[-2]

    # find linear combination such that:
    # a * vect1[0] + b * vect2[0] = m1_set   and
    # a * vect1[1] + b * vect2[1] = m2_set
    ## But this rearranges to:

    coefa = ( vect2[1] * m1_set - m2_set * vect2[0] ) / (vect2[1]*vect1[0] - vect1[1]*vect2[0] )
    coefb = (vect1[1]*m1_set - m2_set *vect1[0] ) /(vect1[1]*vect2[0] - vect2[1]*vect1[0] )

    elements = [coefa*vect1[k]+coefb*vect2[k]  for k in range(len(vect1)) ]
    return elements

elements = elements_assuming_2d(vh)
#elements = [element.real for element in elements if element.imag == 0 ]
syserrors = [syserror(elements[i], vals_set[i]) for i in range(len(elements))]
avg2d, rms2d = combinedsyserror(syserrors, 1)

vals3 = [['m1', 'm2', 'b1', 'b2', 'k1', 'k2','c12', 'Driving Force']]
vals3.append(elements)
vals3.append(vals_set)
vals3.append(syserrors)

#Calculate Rsqrd compared to 2D nullspace to test the 2D nullspace model
M1, M2, B1, B2, K1, K2, K12, FD = elements

plotting = False

amp1_rsqrd2D = rsqrd(model=curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), data=R1_amp, plot=plotting, x=drive)
phase1_rsqrd2D = rsqrd(data=R1_phase, model=theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
amp2_rsqrd2D = rsqrd(data=R2_amp, model=curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)
phase2_rsqrd2D = rsqrd(data = R2_phase, model= theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), plot=plotting, x=drive)

elementsdf3 = pd.DataFrame(vals3, index = ['labels', 'from_svd', 'set', "syserror"])
new_header3 = elementsdf3.iloc[0] 
elementsdf3.columns = new_header3

display(elementsdf3)


print('Average systematic error: ', '{num:.{dig}f}'.format(num=avg1d, dig=3))
print('RMS systematic error: ', '{num:.{dig}f}'.format(num=rms1d, dig=3))

print("How good are the 2D nullspace results?")
print('Noise level: ' + str(noiselevel))
print('R1 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R1.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R1.max(), dig=3))
print('R2 amp SNR: ' + '{num:.{dig}g}'.format(num=df.SNR_R2.min(), dig=3) + ' to ' 
      + '{num:.{dig}g}'.format(num=df.SNR_R2.max(), dig=3))
print('Average systematic error: ', '{num:.{dig}f}'.format(num=avg2d, dig=3),
      'whereas for 1D it was: ', '{num:.{dig}f}'.format(num=avg1d, dig=3))
print('RMS systematic error:   ', '{num:.{dig}f}'.format(num=rms2d, dig=3), 
      ' whereas for 1D it was: ', '{num:.{dig}f}'.format(num=rms1d, dig=3))

fig,ax = plt.subplots()
if labelbarchart:
    bargraph = ax.bar(elementsdf3.columns,elementsdf3.transpose()['syserror']*100)
    plt.gca().bar_label(bargraph)
else:
    (elementsdf3.transpose()['syserror']*100).plot(kind='bar')
plt.gca().set_ylabel('systematic error (%)\n')
plt.gca().set_xlabel('');
plt.title("2D nullspace")
plt.show()

print("Number of frequencies: ", len(p))
print("R1 Amp Rsqrd = ", amp1_rsqrd2D,
     "\nR1 Phase Rsqrd = ", phase1_rsqrd2D,
      "\nR2 Amp Rsqrd = ", amp2_rsqrd2D,
      "\nR2 Phase Rsqrd = ", phase2_rsqrd2D)


In [ ]:
#Plots of singular value decomposition

fig, ((ax1, ax3),(ax2,ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (10,10))

ax1.plot(drive, R1_amp, '.', color = 'lightsteelblue', label='simulated data') # simulated data
ax1.plot(drive, curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black', label='SVD results') # predicted spectrum from SVD)
ax1.set_ylabel('Amplitude\n')
ax1.set_title('Simulated R1 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax1.plot(df.drive[i], df.R1Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)
        
ax2.plot(drive, R1_phase, '.', color = 'lightsteelblue')
ax2.plot(drive, theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
ax2.set_ylabel('Phase (rad)')
ax2.set_title('Simulated R1 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax2.plot(df.drive[i], df.R1Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)
        

ax3.plot(drive, R2_amp, '.', color = 'lightsteelblue')
ax3.plot(drive, curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
ax3.set_ylabel('Amplitude\n')
ax3.set_title('Simulated R2 Amp')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax3.plot(df.drive[i], df.R2Amp[i], 'mo', fillstyle='none', markeredgewidth = 3)

ax4.plot(drive, R2_phase, '.', color = 'lightsteelblue')
ax4.plot(drive, theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
ax4.set_ylabel('Phase (rad)')
ax4.set_title('Simulated R2 Phase')

#For loop to plot R1 amplitude values from table
for i in range(df.shape[0]):
    ax4.plot(df.drive[i], df.R2Phase[i], 'mo', fillstyle='none', markeredgewidth = 3)

for ax in [ax1, ax2, ax3, ax4]:
    plt.sca(ax)
    plt.xticks([res1, res2])
    ax.set_xlabel('Freq (rad/s)')

plotcomplex(Z1, drive, 'Complex Amplitude Z1', ax=ax5, label_markers=[res1,res2])
ax5.scatter(np.real(df.R1AmpCom), np.imag(df.R1AmpCom), s=150, facecolors='none', edgecolors='k') 
plotcomplex(Z2, drive, 'Complex Amplitude Z2', ax=ax6, label_markers=[res1,res2])
ax6.scatter(np.real(df.R2AmpCom), np.imag(df.R2AmpCom), s=150, facecolors='none', edgecolors='k') 

# true curves
morefrequencies = np.linspace(0.1, 5, num = n*10)
ax5.plot(realamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp1(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)
ax6.plot(realamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         imamp2(morefrequencies, k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set, 0), 
         color='gray', alpha = .5)

# svd curves
print("2D nullspace")
morefrequencies = np.linspace(0.1, 5, num = n*10)
ax5.plot(realamp1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), 
         imamp1(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')
ax6.plot(realamp2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), 
         imamp2(morefrequencies, K1, K2, K12, B1, B2, FD, M1, M2, 0), '--', color='black')

plt.tight_layout()

In [ ]:
#stophere

**Sweep through possible pairs of frequencies**

In [ ]:
#Code that loops through points of different spacing

#initiate arrays for badness of fit values
fits = []

# initate array for systematic error sum of squares values
results = []

# Loop over possible combinations of frequency indices, i1 and i2
for i1 in range(n):
    freq1 = drive[i1]
   
    for i2 in range(n):
        freq2 = drive[i2]
        
        ## Recalculate the noise for each pair. (I checked: Python doesn't cache the function returns.)
        # Values for frequency 1
        na11,np11, Z1_1 = noisyR1ampphase(freq1)
        na21,np21, Z2_1 = noisyR2ampphase(freq1)
        na11 = na11[0] # get rid of the arrays; just want raw number
        np21 = np21[0]
        na21 = na21[0]
        Z1_1 = Z1_1[0]
        Z2_1 = Z2_1[0]
        
        # Values for frequency 2
        na12,np12, Z1_2 = noisyR1ampphase(freq2)
        na22,np22, Z2_2 = noisyR2ampphase(freq2)
        na12 = na12[0]
        np12 = np12[0]
        na22 = na22[0]
        np22 = np22[0]
        Z1_2 = Z1_2[0]
        Z2_2 = Z2_2[0]

        columnnames = ['drive', 'R1Amp', 'R1Phase', 'R2Amp', 'R2Phase', 'R1AmpCom', 'R2AmpCom']
        simulateddata = [[freq1, na11, np11, na21,np21, Z1_1, Z2_1],
                         [freq2, na12,np12, na22,np22, Z1_2, Z2_2]]
        measurementdf = pd.DataFrame(data = simulateddata, columns = columnnames)

        Zmatrix = Zmatrix2resonators(measurementdf, 
                                     frequencycolumn = 'drive', 
                                     complexamplitude1 = 'R1AmpCom', 
                                     complexamplitude2 = 'R2AmpCom',
                                     dtype = complex) # storing it as complex avoids a runtime warning
        
        #display(pd.DataFrame(Zmatrix))

        #SVD,  u and vh are 2D unitary arrays and s is a 1D array of the input matrix's singular values
        u, s, vh = np.linalg.svd(Zmatrix, full_matrices = True)

        # extract parameters found by SVD
        #assign variables # mass of resonator 1, mass 2, damping 1, damping 2, stiffness 1, stiffness 2, coupling stiffness, force
        [M1, M2, B1, B2, K1, K2, K12, FD] = vh[-1] # the 7th singular value is the smallest one (closest to zero)

        # normalize elements vector to the force, assuming 1D nullspace
        M1, M2, B1, B2, K1, K2, K12, FD = normalize_elements_by_force(M1, M2, B1, B2, K1, K2, K12, FD, F_set)
                
        # Alternative check: 
        # calculate how close the SVD-determined parameters are compared to the originally set parameters

        el = [M1, M2, B1, B2, K1, K2, K12, FD]
        syserrors = [syserror(el[i], vals_set[i]) for i in range(len(el))]
     
        # Values to compare:
        # Set values: k1_set, k2_set, k12_set, b1_set, b2_set, F_set, m1_set, m2_set
        # SVD-determined values: M1, M2, B1, B2, K1, K2, K12, FD
        K1syserror = syserror(K1,k1_set)
        K2syserror = syserror(K2,k2_set)
        K12syserror = syserror(K12,k12_set)
        B1syserror = syserror(B1,b1_set)
        B2syserror = syserror(B2,b2_set)
        Fsyserror = syserror(FD,F_set)
        m1syserror = syserror(M1,m1_set)
        m2syserror = syserror(M2,m2_set)
        avgsyserror, rmserror = combinedsyserror(syserrors,2) # subtract 2 degrees of freedom for 2D nullspace
        """        parametererror2 = np.sqrt((K1syserror**2 + K2syserror**2 + K12syserror**2 \
                                  + B1syserror**2 + B2syserror**2 + Fsyserror**2 + \
                                  m1syserror**2 + m2syserror**2)/8)"""
        results.append([drive[i1],
                          drive[i2],
                          drive[i2] - drive[i1],
                          M1, M2, B1, B2, K1, K2, K12, FD,
                          K1syserror,
                          K2syserror,
                          K12syserror,
                          B1syserror,
                          B2syserror,
                          Fsyserror,
                          m1syserror,
                          m2syserror,
                          avgsyserror, rmserror,
                         np.sum(R1_amp - curve1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2, 
                        np.sum((R1_phase - theta1(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                        np.sum((R2_amp - curve2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                        np.sum((R2_phase - theta2(drive, K1, K2, K12, B1, B2, FD, M1, M2, 0))**2),
                        s[-1],s[-2],
                       R1_amp_noiseless[i1], R1_phase_noiseless[i1], R2_amp_noiseless[i1], R2_phase_noiseless[i1],
                       R1_amp_noiseless[i2], R1_phase_noiseless[i2], R2_amp_noiseless[i2], R2_phase_noiseless[i2],
                       R1_phase_noiseless[i2]-R1_phase_noiseless[i1]])
        

resultsdf = pd.DataFrame(
    data=results, 
    columns = ['Freq1','Freq2', 'Difference',
        'M1', 'M2', 'B1', 'B2', 'K1', 'K2', 'K12', 'FD',
        'K1syserror',
        'K2syserror',
        'K12syserror',
        'B1syserror',
        'B2syserror',
        'Fsyserror',
        'm1syserror',
        'm2syserror',
        'avgsyserror', 'rmserror','R1AmpFit', 'R1PhaseFit', 'R2AmpFit', 'R2PhaseFit',
        'smallest singular value', 'second smallest singular value',
        'R1_amp_noiseless_a', 'R1_phase_noiseless_a', 'R2_amp_noiseless_a', 'R2_phase_noiseless_a',
        'R1_amp_noiseless_b', 'R1_phase_noiseless_b', 'R2_amp_noiseless_b', 'R2_phase_noiseless_b', 'R1_phase_diff'])

In [ ]:
(resultsdf[100:105].transpose())

In [ ]:
figsize = (12,10/1.3)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=figsize)

print('Assuming 1D nullspace (where is this assumption ok?)')
print('Noiselevel: ' + str(noiselevel))

plt.sca(ax1)
lambdagrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'smallest singular value').sort_index(axis = 0, ascending = False)
myheatmap(lambdagrid, "smallest singular value", cmap = 'Greys'); 

plt.sca(ax2)
lambda2grid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'second smallest singular value').sort_index(axis = 0, ascending = False)
myheatmap(lambda2grid, "2nd smallest\nsingular value", cmap = 'viridis'); 

plt.sca(ax3)
errgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'm2syserror').sort_index(axis = 0, ascending = False)
myheatmap(errgrid, "m2 sys error", vmax=0.5, cmap='magma_r'); 

plt.sca(ax4)
SSgrid=resultsdf.pivot_table(index = 'Freq1', columns = 'Freq2', values = 'avgsyserror').sort_index(axis = 0, ascending = False)
myheatmap(SSgrid, "average sys error",  vmax=0.5, cmap='magma_r'); 

for ax in [ax1,ax2,ax3,ax4]:
    plt.sca(ax)
    ax.axis('equal');
    plt.xticks([res1, res2])
    plt.yticks([res1, res2])

fig.tight_layout()

In [ ]:
## Plot systematic error of parameters, sum of squares, as a function of the higher freq 

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(resultsdf.Freq2,resultsdf.avgsyserror, '.')
ax1.set_xlabel('Frequency 2 (rad/s)')
ax1.set_ylabel('Average Sys error')
#ax1.set_title('Systematic error sum of squares')



ax2.plot(resultsdf.Freq2,(resultsdf.M1), '.')
ax2.set_ylabel('Mass 1')
ax2.set_xlabel('Frequency 2 (rad/s)')

ax3.plot(resultsdf.Freq2,(resultsdf.M2), '.')
ax3.set_ylabel('Mass 2')
ax3.set_xlabel('Frequency 2 (rad/s)')

ax4.plot(resultsdf.Freq2,(resultsdf.B2), '.')
ax4.set_ylabel(' value of damping 2')
ax4.set_xlabel('Frequency 2 (rad/s)')

plt.tight_layout()


In [ ]:
#Plots of badness of fits as a function of separation between points

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize = (14,10))

ax1.plot(resultsdf.Difference, (resultsdf.R1AmpFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('R1 Amp Badness of Fit')
        
ax2.plot(resultsdf.Difference, (resultsdf.R1PhaseFit), '.', color = 'silver')
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('R1 Phase Badness of Fit')     

ax3.plot(resultsdf.Difference, (resultsdf.R2AmpFit), '.', color = 'lightsteelblue')
ax3.set_xlabel('Frequency Difference (rad/s)')
ax3.set_ylabel('Difference Squared ($m^2$)')
ax3.set_title('R2 Amp Badness of Fit')
    
ax4.plot(resultsdf.Difference, (resultsdf.R2PhaseFit), '.', color = 'lightsteelblue')
ax4.set_xlabel('Frequency Difference (rad/s)')
ax4.set_ylabel('Difference Squared ($rad^2$)')
ax4.set_title('R2 Phase Badness of Fit')
        
plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("R1 Amp min = ", resultsdf.iloc[resultsdf.R1AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", resultsdf.iloc[resultsdf.R1PhaseFit.idxmin()].Difference, "rad",
     "\nR2 Amp min = ", resultsdf.iloc[resultsdf.R2AmpFit.idxmin()].Difference, "m",
     "\nR2 Phase min = ", resultsdf.iloc[resultsdf.R2PhaseFit.idxmin()].Difference, "rad")

In [ ]:
#Plot combined amplitude and phase Badness of Fit parameters

#Plots of badness of fits as a function of seperation between points

fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize = (14,5))

ax1.plot(resultsdf.Difference, (resultsdf.R1AmpFit)+(resultsdf.R2AmpFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Difference Squared ($m^2$)')
ax1.set_title('Amplitude Badness of Fit')
        
ax2.plot(resultsdf.Difference, (resultsdf.R1PhaseFit)+(resultsdf.R2PhaseFit), '.', color = 'silver')
ax2.set_xlabel('Frequency Difference (rad/s)')
ax2.set_ylabel('Difference Squared ($rad^2$)')
ax2.set_title('Phase Badness of Fit')     

plt.tight_layout()

#Print frequency difference with smallest badness of fit

print("Amp min = ", resultsdf.iloc[resultsdf.R1AmpFit.idxmin()].Difference + 
      resultsdf.iloc[resultsdf.R2AmpFit.idxmin()].Difference, "m",
     "\nR1 Phase min = ", resultsdf.iloc[resultsdf.R1PhaseFit.idxmin()].Difference +
      resultsdf.iloc[resultsdf.R2PhaseFit.idxmin()].Difference, "rad")

In [ ]:
#Plots of badness of fits as a function of seperation between points

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(resultsdf.Difference, np.log(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit), '.', color = 'silver')
ax1.set_xlabel('Frequency Difference (rad/s)')
ax1.set_ylabel('Log Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

print("Total min = ", resultsdf.iloc[(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit).idxmin()].Difference, "rad/s")

In [ ]:
## Plots of badness of fits as a function of seperation between points

fig, (ax1) = plt.subplots(1, 1, figsize = (8,6))

ax1.plot(resultsdf.R1_phase_diff, np.log(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit), '.', color = 'silver')
ax1.set_xlabel('Phase Difference (rad)')
ax1.set_ylabel('Log Difference Squared ($m^2$)')
ax1.set_title('Total Badness of Fit')

print("Total min = ", resultsdf.iloc[(resultsdf.R1AmpFit + resultsdf.R2AmpFit + resultsdf.R1PhaseFit + resultsdf.R2PhaseFit).idxmin()].Difference, "rad/s")